# Roll control analysis Pt. 2 
*by William Harrington*  
*Portland State Aerospace Society*  

In the last notebook, I looked at the velocity and altitude curves plus the angular position, angular velocity, and angular acceleration of the Launch-12 flight data.

I found that the average angular acceleration during the flight was 0. This means that the roll control system was successful in keeping the roll rate (angular velocity) relatively constant. Therefore, the derivative of this data (angular acceleration) would be 0 on average. From this I have concluded that in order to keep the roll rate to 0, as was originally intended, the PID loop should of been designed for controlling roll angle and not roll rate.
                 
This time I want to take a look at the output of the PID loop so I can examine how the control loop was performing during flight. The control loop was designed to use the output of the PID to estimate an angular position for the canards so they can create a torque that opposes the one being imposed on the rocket. The angular position is clamped to +/- 15 degrees. A request for that angular position is then sent to the Flight Computer. It is important to note that there is a latency here that we assume is around 3ms. The code that simulates this returns to us the current angular position of the canards, and every 3ms accepts the angular position that was sent. The current angular position of the canards tells us the angular acceleration being caused by the canards.

I will need the accelerometer and gyro readings to do this. I can use the accelerometer readings to get altitude and velocity which are important parameters in our mathematical model that describes the angular acceleration caused by the 4 canards. Then I can use the gyro readings by feeding them to the PID loop and observing the output.  

The gyro readings are noisy so I will need to eliminate some of that noise to make sense of what is going on. A lowpass filter should do the trick.  

In [ ]:
# lets get everything set up
%matplotlib inline
from matplotlib import rcParams
rcParams.update({'font.size': 16})

# stuff from before, supports math stuff and plotting
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.signal import savgol_filter

# other stuff we will need for this code
# code from the roll-control repository
from PIDcontroller import PIDController  # PID controller, modified to return output from each stage
import lv2  # launch vehicle behavior

# extract data
columns = np.loadtxt('fc-data/ADIS.csv', delimiter=',', unpack=True)

time = columns[1]  # time
gyro_x = columns[3]  # x-axis gyro readings
acc_x = columns[6]  # x-axis accelerometer readings

columns = np.loadtxt("fc-data/ROLL.csv", delimiter=',', unpack=True)

# actual fin angle during flight
fin_angle = columns[2]

# umblicial disconnect event
t_0 = 117853569585227

# element wise subtraction
time = np.subtract(time, t_0)

# convert from ns to s
time = np.divide(time, 1e9)

# get rid of unwanted readings
time = time[3650:28125]
gyro_x = gyro_x[3650:28125]
acc_x = acc_x[3650:28125]
fin_angle = fin_angle[3650:28125]

# subtract gravity from accelerometer readings
acc_x = np.subtract(acc_x, 9.8)

# filter gyro readings
acc_x_savgol = savgol_filter(acc_x, 201, 2)
acc_x = acc_x_savgol

# filter gyro readings
gyro_x_savgol = savgol_filter(gyro_x, 201, 2)

# now lets look at the result
fig, ax10 = plt.subplots(nrows=1, sharex=True, figsize=(16,9))
ax10.plot(time, gyro_x, color='grey', alpha=.15, label='Unfiltered')
ax10.plot(time, gyro_x_savgol, color='red', label='Filtered')
ax10.set_title('Filtered gyro readings')
ax10.set_ylabel('Angular velocity [$deg/s$]')
ax10.set_xlabel('Time [$s$]')
ax10.legend(loc=1, fontsize='x-small')
plt.show()

In [ ]:
# differentiate gyro readings to get angular acceleration
ang_acc = np.diff(gyro_x)
ang_acc *= 819.2  # sample rate

# filter derivative of gyro readings
angacc_savgol = savgol_filter(gyro_x, 201, 2, deriv=1, delta=1/819.2)

# now lets look at the result
fig, ax11 = plt.subplots(nrows=1, sharex=True, figsize=(16,9))
ax11.plot(time[1:], ang_acc, color='grey', alpha=.15, label='Unfiltered')
ax11.plot(time, angacc_savgol, color='red', label='Filtered')
ax11.set_title('Angular acceleration of the rocket')
ax11.set_ylabel('Angular acceleration [$deg/s^2$]')
ax11.set_xlabel('Time [$s$]')
ax11.set_ylim([-500, 500])
ax11.legend(loc=1, fontsize='x-small')
plt.show()

ang_acc_np = np.array(ang_acc)

print 'Average filtered angular acceleration: %3.2f deg/s^2' % ang_acc_np.mean()

In [ ]:
# use those sexy filtered readings
gyro_x = gyro_x_savgol
ang_acc = angacc_savgol

# get velocity from integrating accelerometer
velocity = []
for i in range(len(acc_x)):
    # avoid integration error
    if(i > 0):
        # use simpsons rule on the samples to integrate
        velocity.append(simps(acc_x[:i], time[:i]))
        
# get position from integrating velocity
position = []
for j in range(len(velocity)):
    # avoid integration error
    if(j > 0):
        # use simpsons rule on the samples to integrate
        position.append(simps(velocity[:j], time[:j]))

# now that we've got that
# time to set up PID controller
pid = PIDController(p=5, i=.01, d=0)
pid.setTarget(0.0)

# for storing predicted canard angle
canard_angle = []

# for storing PID outputs
pids = []
prop_stage = []
integral_stage = []
derivative_stage = []

# for storing angular acc caused by canard
canard_ang_acc = []  # predicted
actual_canard_ang_acc = []  # actual

# iterating over len(time)-2, since we lose 2 samples in the double integration
for i in range(len(time[2:])):
    
    # step PID by feeding in the gyro readings from the flight
    correction = pid.step(gyro_x[i])
    
    # store correction
    # had to flip signs to match data... why??
    pids.append(-1*correction[0])  # total
    prop_stage.append(-1*correction[1])  # proportional
    integral_stage.append(-1*correction[2])  # integral
    derivative_stage.append(-1*correction[3])  # derivative
    
    
    # estimate alpha from current position, velocity, time
    # had to flip signs to match data... why??
    a = -1*lv2.estimate_alpha(correction[0], position[i], velocity[i], time[i])    
    
    # request current canard angle
    # accepts manuever every 3ms
    a = lv2.servo(a, time[i])
    
    # compute angular acceleration being caused by canard
    aa = lv2.angular_accel(a, position[i], velocity[i], time[i])
    actual_aa = lv2.angular_accel(fin_angle[i], position[i], velocity[i], time[i])
    
    # angular position of canard 
    # and canard angular acceleration
    canard_angle.append(a)
    canard_ang_acc.append(aa)
    actual_canard_ang_acc.append(actual_aa)

### Canard Angle

Whew ok now that's over lets look at what the hell this thing was doing!

First lets glance at the canard angle.  

In [ ]:
# filter canard angle readings
fin_angle = savgol_filter(fin_angle, 201, 2)

fig, ax0 = plt.subplots(nrows=1, sharex=True, figsize=(16,9))
ax0.plot(time[2:], canard_angle, label='Predicted')
ax0.plot(time[2:], fin_angle[2:], label='Actual')
ax0.set_title('Canard angle')
ax0.set_ylabel('Angle [$degrees$]')
ax0.set_xlabel('Time [$s$]')
ax0.set_xlim([0, 30])
ax0.set_ylim([-16, 16])
ax0.legend(loc=1, fontsize='x-small')
plt.show()

Seems as though our algorithm worked as intended.
### PID controller output
Now lets look at the outputs given by the PID controller. We didn't save the output of the PID controller during the flight so there is nothing for us to reference here. However, it is reasonable to believe that feeding the gyro readings through the PID controller code would put out the same thing.

In [ ]:
fig, ax1 = plt.subplots(nrows=1, sharex=True, figsize=(16,9))

ax1.plot(time[2:], pids, label='Total')
ax1.plot(time[2:], prop_stage, label='Proportional')
ax1.plot(time[2:], integral_stage, label='Integral')
ax1.plot(time[2:], derivative_stage, label='Derivative')
ax1.legend(loc=3, fontsize='x-small')
ax1.set_title('Output of PID stages')
ax1.set_ylabel('Output')
ax1.set_xlabel('Time [$s$]')

plt.show()

prop_stage = np.array(prop_stage)
integral_stage = np.array(integral_stage)

# note: we didn't use a derivative stage so it is always 0

print 'Average output of proportional stage: %3.2f' % prop_stage.mean()
print 'Average output of integral stage: %3.2f' % integral_stage.mean()

As expected, the proportional stage does the heavy lifting, making up the majority of the total output of the PID. The integrator quickly saturates and remains constant for the rest of the flight.


### Deriving unknown angular acceleration

The total angular acceleration on the rocket has to be equal to the sum of angular acceleration caused by the canards and some unknown angular acceleration being imposed on the rocket.  
i.e. ang_acc = canard_ang_acc + unknown_ang_acc  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;unknown_ang_acc = ang_acc - canard_ang_acc

In [ ]:
# filter data
actual_canard_ang_acc = savgol_filter(actual_canard_ang_acc, 201, 2)

# array for storing result
unknown_ang_acc = []   

# iterate over time
for q in range(len(time[2:])):
    unknown_ang_acc.append(ang_acc[q]-actual_canard_ang_acc[q])

fig, ax3 = plt.subplots(nrows=1, sharex=True, figsize=(16,9))

ax3.plot(time, ang_acc, label='total')
ax3.plot(time[2:], actual_canard_ang_acc, label='canard')
ax3.plot(time[2:], unknown_ang_acc, label='unknown')
ax3.set_title('Derived unknown Angular acceleration')
ax3.set_ylabel('Angular acceleration [$deg/s^2$]')
ax3.set_xlabel('Time [$s$]')
ax3.legend(loc=1, fontsize='x-small')

plt.show()

### Angular acceleration caused by canards

In [ ]:
fig, ax2 = plt.subplots(nrows=1, sharex=True, figsize=(16,9))

ax2.plot(time[2:], canard_ang_acc, label='predicted')
ax2.plot(time[2:], actual_canard_ang_acc, label='actual')
ax2.set_title('Angular acceleration caused by canards')
ax2.set_ylabel('Angular acceleration [$deg/s^2$]')
ax2.set_xlabel('Time [$s$]')
ax2.legend(loc=1, fontsize='x-small')

plt.show()

### How does the unknown angular acceleration scale with velocity?

One of the assumptions made in simulating flight was that the unknown angular acceleration scales with velocity because the damping force of a spring oscillator is normally linearly dependent on velocity.

Let's see how they compare.

In [ ]:
fig, ax4 = plt.subplots(nrows=1, sharex=True, figsize=(16,9))

ax4.plot(time[2:], velocity[1:], label='z-axis velocity (m/s)')
ax4.plot(time[2:], np.multiply(unknown_ang_acc, -1), label='unknown ang acc (deg/s^2)')
ax4.legend(loc=2, fontsize='x-small')
ax4.set_title('Velocity compared to angular acceleration')
ax4.set_xlabel('Time [$s$]')

plt.show()

Doesn't seem to scale at all. This is absolutely the worst case scenario that we envisioned.